# Generar CSV

In [40]:
import numpy as np 
import pandas as pd 

In [41]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

print("Abriendo ventana de selección...")

root = tk.Tk()
root.withdraw() 

ruta_archivo = filedialog.askopenfilename(
    title="Selecciona tu Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_consolidado = pd.read_excel(
            ruta_archivo, 
            sheet_name='SERVICIOS', 
            header=3
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_consolidado)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_consolidado = df_consolidado.loc[:, ~df_consolidado.columns.str.contains('^Unnamed')]

        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        if columna_clave in df_consolidado.columns:
            filas_antes = len(df_consolidado)
            df_consolidado = df_consolidado.dropna(subset=[columna_clave])
            filas_despues = len(df_consolidado)
            print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_consolidado.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_consolidado[col].dtype == 'object':
                df_consolidado[col] = (
                    df_consolidado[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_consolidado[col] = pd.to_numeric(df_consolidado[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_consolidado.head())
        print(df_consolidado.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

Abriendo ventana de selección...
🚀 Cargando archivo: C:/Users/Usuario/Desktop/Scripts_automatización/Asia_BI/REPORTE_FINANZAS_2026113_142550_SEM2.xlsx
✅ Lectura inicial completada. Filas detectadas: 3111

📊 DATOS LISTOS:
   IdServicio  IdConductor   Flota   Membresía  IdMovil PlacaAuto ModeloAuto  \
0     5453691         2808  Urbano  PARTNER V1     2808    CJV095    Xpander   
1     5453712         9740  Urbano  PARTNER V1     9740    F7M239    COROLLA   
2     5453714         5023  Urbano        FULL     5023    CPR599     SOLUTO   
3     5453711         2218  Urbano    FLEXIBLE     2218    CVN594        X70   
4     5453715         3198  Urbano   REGULAR 1     3198    CRG272        X70   

                          Conductor Fecha_Servicio Hora_Servicio  ...  \
0       DAVID MICHAEL VARGAS DÁVILA     2026-01-09      10:15:00  ...   
1       JORGE LUIS ESTELA COARICONA     2026-01-09      05:30:00  ...   
2               SAUL HUARIS BENDEZU     2026-01-09      05:20:00  ...   
3  AND

In [42]:
df_consolidado.head()

,IdServicio,IdConductor,Flota,Membresía,IdMovil,PlacaAuto,ModeloAuto,Conductor,Fecha_Servicio,Hora_Servicio,...,Maletera,FacturaEnvio,NroDocumentoEnvio,Ticket Counter,Total_Comisionable,Total_NoComisionable,Pago a Conductor,Comisión Conductor,Comisión Directo,Importe Factura
0,5453691,2808,Urbano,PARTNER V1,2808,CJV095,Xpander,DAVID MICHAEL VARGAS DÁVILA,2026-01-09,10:15:00,...,0.0,False,(ERROR),NaN,20.9,0.0,20.9,1,0,20.9
1,5453712,9740,Urbano,PARTNER V1,9740,F7M239,COROLLA,JORGE LUIS ESTELA COARICONA,2026-01-09,05:30:00,...,0.0,False,(ERROR),NaN,27.6,0.0,27.6,1,0,27.6
2,5453714,5023,Urbano,FULL,5023,CPR599,SOLUTO,SAUL HUARIS BENDEZU,2026-01-09,05:20:00,...,0.0,False,(ERROR),NaN,64.7,0.0,64.7,1,0,64.7
3,5453711,2218,Urbano,FLEXIBLE,2218,CVN594,X70,ANDERSON RENATO CALLAÑAUPA AYALA,2026-01-09,05:15:00,...,0.0,False,(ERROR),NaN,106.5,0.0,106.5,1,0,106.5
4,5453715,3198,Urbano,REGULAR 1,3198,CRG272,X70,ENZO ISRAEL MICUNCO ROMERO,2026-01-09,05:40:00,...,0.0,False,(ERROR),NaN,8.0,0.0,8.0,1,0,8.0


In [43]:
df_consolidado.tail()

,IdServicio,IdConductor,Flota,Membresía,IdMovil,PlacaAuto,ModeloAuto,Conductor,Fecha_Servicio,Hora_Servicio,...,Maletera,FacturaEnvio,NroDocumentoEnvio,Ticket Counter,Total_Comisionable,Total_NoComisionable,Pago a Conductor,Comisión Conductor,Comisión Directo,Importe Factura
3106,5457318,9364,Urbano,PARTNER V1,9364,CCO340,GLORY 560,JOSÉ LUIS CAMARGO LLAMOCA,2026-01-11,21:45:00,...,0.0,True,B001-416008,NaN,26.4,0.0,26.4,1,0,26.4
3107,5456440,2367,Urbano,FLEXIBLE,2367,CAV690,GLORY 580,LUIS ALFREDO DIAZ QUIROZ,2026-01-11,01:36:46,...,0.0,True,B001-415874,NaN,42.0,0.0,42.0,1,0,17.0
3108,5452699,2168,Urbano,FULL,2168,CBE341,ETIOS,RONALD MICHAEL HAZELL ALCANTARA,2026-01-11,15:30:00,...,0.0,True,B001-415971,NaN,87.0,0.0,87.0,1,0,87.0
3109,5456988,3198,Urbano,REGULAR 1,3198,CRG272,X70,ENZO ISRAEL MICUNCO ROMERO,2026-01-11,19:00:59,...,0.0,True,F001-106369,NaN,98.4,0.0,98.4,1,0,98.4
3110,5456628,1891,Aeropuerto,REGULAR 1,1891,CRD692,HONOR,LINO FELICITO MIRANDA ARTEAGA,2026-01-11,13:30:00,...,0.0,False,Pendiente(ERROR),NaN,50.0,0.0,50.0,1,0,50.0


In [44]:
len(df_consolidado.columns)

263

In [45]:
columnas_necesarias = ['IdServicio','IdMovil','Fecha_Servicio','Hora_Servicio','Total_Servicio','NombreComercial','NombreCompleto','DescModoReserva','DescTipoPago','DirOrigen','ZonaOrigen','DistritoOrigen','Destino_1','ZonaDestino_1','DistritoDestino_1','Zonas','UNIDAD_NEGOCIO_DIRECTO','Importe Factura', 'Total_Peaje','Total_Parqueo']
df_consolidado=df_consolidado[columnas_necesarias]
df_consolidado.head()

,IdServicio,IdMovil,Fecha_Servicio,Hora_Servicio,Total_Servicio,NombreComercial,NombreCompleto,DescModoReserva,DescTipoPago,DirOrigen,ZonaOrigen,DistritoOrigen,Destino_1,ZonaDestino_1,DistritoDestino_1,Zonas,UNIDAD_NEGOCIO_DIRECTO,Importe Factura,Total_Peaje,Total_Parqueo
0,5453691,2808,2026-01-09,10:15:00,20.9,ALICORP S.A.A.,REINA ANDREA MEJIA AGUILA,App,Empresa,Avenida Paseo De La República 2199,LA VICTORIA SANTA CATALINA,LA VICTORIA,Universidad de Lima,SURCO JOCKEY,SANTIAGO DE SURCO,SURCO JOCKEY,URBANO,20.9,0.0,0
1,5453712,9740,2026-01-09,05:30:00,27.6,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,LA VICTORIA SANTA CATALINA,LA VICTORIA,"AV. SIMÓN BOLÍVAR 2150, PUEBLO LIBRE, PERÚ",PUEBLO LIBRE,PUEBLO LIBRE,PUEBLO LIBRE,URBANO,27.6,0.0,0
2,5453714,5023,2026-01-09,05:20:00,64.7,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,LA VICTORIA SANTA CATALINA,LA VICTORIA,AV A MZ 38 LT 3 ALT COLEGIO ABRAHAM BALDELOMAR,LOS OLIVOS PRO,LOS OLIVOS,LOS OLIVOS PRO,URBANO,64.7,0.0,0
3,5453711,2218,2026-01-09,05:15:00,106.5,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,LA VICTORIA SANTA CATALINA,LA VICTORIA,"JIRON LAS MAGNOLIAS, CARABAYLLO, PERÚ",CARABAYLLO SANTA MARIA,CARABAYLLO,CARABAYLLO SANTA MARIA,URBANO,106.5,0.0,0
4,5453715,3198,2026-01-09,05:40:00,8.0,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,LA VICTORIA SANTA CATALINA,LA VICTORIA,DESPLAZAMIENTO,DESPLAZAMIENTO A,LIMA,DESPLAZAMIENTO A,URBANO,8.0,0.0,0


In [46]:
len(df_consolidado.columns)

20

In [47]:
df_consolidado['Total_ventas'] = df_consolidado['Total_Servicio'] - df_consolidado['Total_Peaje'] - df_consolidado['Total_Parqueo']
df_consolidado.head()

,IdServicio,IdMovil,Fecha_Servicio,Hora_Servicio,Total_Servicio,NombreComercial,NombreCompleto,DescModoReserva,DescTipoPago,DirOrigen,...,DistritoOrigen,Destino_1,ZonaDestino_1,DistritoDestino_1,Zonas,UNIDAD_NEGOCIO_DIRECTO,Importe Factura,Total_Peaje,Total_Parqueo,Total_ventas
0,5453691,2808,2026-01-09,10:15:00,20.9,ALICORP S.A.A.,REINA ANDREA MEJIA AGUILA,App,Empresa,Avenida Paseo De La República 2199,...,LA VICTORIA,Universidad de Lima,SURCO JOCKEY,SANTIAGO DE SURCO,SURCO JOCKEY,URBANO,20.9,0.0,0,20.9
1,5453712,9740,2026-01-09,05:30:00,27.6,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,LA VICTORIA,"AV. SIMÓN BOLÍVAR 2150, PUEBLO LIBRE, PERÚ",PUEBLO LIBRE,PUEBLO LIBRE,PUEBLO LIBRE,URBANO,27.6,0.0,0,27.6
2,5453714,5023,2026-01-09,05:20:00,64.7,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,LA VICTORIA,AV A MZ 38 LT 3 ALT COLEGIO ABRAHAM BALDELOMAR,LOS OLIVOS PRO,LOS OLIVOS,LOS OLIVOS PRO,URBANO,64.7,0.0,0,64.7
3,5453711,2218,2026-01-09,05:15:00,106.5,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,LA VICTORIA,"JIRON LAS MAGNOLIAS, CARABAYLLO, PERÚ",CARABAYLLO SANTA MARIA,CARABAYLLO,CARABAYLLO SANTA MARIA,URBANO,106.5,0.0,0,106.5
4,5453715,3198,2026-01-09,05:40:00,8.0,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,LA VICTORIA,DESPLAZAMIENTO,DESPLAZAMIENTO A,LIMA,DESPLAZAMIENTO A,URBANO,8.0,0.0,0,8.0


In [48]:

df_consolidado['destinoFinal'] = df_consolidado['Zonas'].str.split('/').str[-1].str.strip()

print(df_consolidado[['Zonas', 'destinoFinal']].head())

                    Zonas            destinoFinal
0            SURCO JOCKEY            SURCO JOCKEY
1            PUEBLO LIBRE            PUEBLO LIBRE
2          LOS OLIVOS PRO          LOS OLIVOS PRO
3  CARABAYLLO SANTA MARIA  CARABAYLLO SANTA MARIA
4        DESPLAZAMIENTO A        DESPLAZAMIENTO A


In [49]:
df_consolidado.head()

,IdServicio,IdMovil,Fecha_Servicio,Hora_Servicio,Total_Servicio,NombreComercial,NombreCompleto,DescModoReserva,DescTipoPago,DirOrigen,...,Destino_1,ZonaDestino_1,DistritoDestino_1,Zonas,UNIDAD_NEGOCIO_DIRECTO,Importe Factura,Total_Peaje,Total_Parqueo,Total_ventas,destinoFinal
0,5453691,2808,2026-01-09,10:15:00,20.9,ALICORP S.A.A.,REINA ANDREA MEJIA AGUILA,App,Empresa,Avenida Paseo De La República 2199,...,Universidad de Lima,SURCO JOCKEY,SANTIAGO DE SURCO,SURCO JOCKEY,URBANO,20.9,0.0,0,20.9,SURCO JOCKEY
1,5453712,9740,2026-01-09,05:30:00,27.6,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,"AV. SIMÓN BOLÍVAR 2150, PUEBLO LIBRE, PERÚ",PUEBLO LIBRE,PUEBLO LIBRE,PUEBLO LIBRE,URBANO,27.6,0.0,0,27.6,PUEBLO LIBRE
2,5453714,5023,2026-01-09,05:20:00,64.7,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,AV A MZ 38 LT 3 ALT COLEGIO ABRAHAM BALDELOMAR,LOS OLIVOS PRO,LOS OLIVOS,LOS OLIVOS PRO,URBANO,64.7,0.0,0,64.7,LOS OLIVOS PRO
3,5453711,2218,2026-01-09,05:15:00,106.5,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,"JIRON LAS MAGNOLIAS, CARABAYLLO, PERÚ",CARABAYLLO SANTA MARIA,CARABAYLLO,CARABAYLLO SANTA MARIA,URBANO,106.5,0.0,0,106.5,CARABAYLLO SANTA MARIA
4,5453715,3198,2026-01-09,05:40:00,8.0,EXSA,LLULLUY COYCAPUZA MARIA ISABEL,Correo,Empresa,CRUZ DEL SUR AV. JAVIER PRADO ESTE # 1109,...,DESPLAZAMIENTO,DESPLAZAMIENTO A,LIMA,DESPLAZAMIENTO A,URBANO,8.0,0.0,0,8.0,DESPLAZAMIENTO A


In [50]:
df_consolidado.to_csv('df_consolidado.csv', mode = 'a', header = False, index=False, encoding='utf-8-sig', sep=',')

# Importación de membresía

In [68]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

print("Abriendo ventana de selección...")

root = tk.Tk()
root.withdraw() 

ruta_archivo = filedialog.askopenfilename(
    title="Selecciona tu Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_membresia = pd.read_excel(
            ruta_archivo, 
            sheet_name='RESUMEN GENERAL', 
            header=5
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_membresia)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        df_membresia = df_membresia.loc[:, ~df_membresia.columns.str.contains('^Unnamed')]

        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        if columna_clave in df_membresia.columns:
            filas_antes = len(df_membresia)
            df_membresia = df_membresia.dropna(subset=[columna_clave])
            filas_despues = len(df_membresia)
            print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_membresia.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_membresia[col].dtype == 'object':
                df_membresia[col] = (
                    df_membresia[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_membresia[col] = pd.to_numeric(df_membresia[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_membresia.head())
        print(df_membresia.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

Abriendo ventana de selección...
🚀 Cargando archivo: C:/Users/Usuario/Desktop/Scripts_automatización/Asia_BI/Fuentes/LIQ_ASIA_#2.xlsx
✅ Lectura inicial completada. Filas detectadas: 36

📊 DATOS LISTOS:
   #Móvil  S/ Cuota     Tipo
0  3504.0     350.0  PREMIUN
1  7474.0     350.0  PREMIUN
2  3788.0     350.0  PREMIUN
3  4027.0     350.0  PREMIUN
4  9172.0     350.0  PREMIUN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   #Móvil    26 non-null     float64
 1   S/ Cuota  27 non-null     float64
 2   Tipo      7 non-null      object 
dtypes: float64(2), object(1)
memory usage: 996.0+ bytes
None


In [69]:
df_membresia.head()

,#Móvil,S/ Cuota,Tipo
0,3504.0,350.0,PREMIUN
1,7474.0,350.0,PREMIUN
2,3788.0,350.0,PREMIUN
3,4027.0,350.0,PREMIUN
4,9172.0,350.0,PREMIUN


In [70]:
df_membresia.rename(columns={'#Móvil': 'Movil', 'S/ Cuota': 'Cuota'}, inplace=True)
df_membresia.head()

,Movil,Cuota,Tipo
0,3504.0,350.0,PREMIUN
1,7474.0,350.0,PREMIUN
2,3788.0,350.0,PREMIUN
3,4027.0,350.0,PREMIUN
4,9172.0,350.0,PREMIUN


In [71]:
df_membresia.tail()

,Movil,Cuota,Tipo
31,NaN,NaN,NaN
32,NaN,NaN,NaN
33,NaN,NaN,NaN
34,NaN,NaN,NaN
35,NaN,4230.0,NaN


In [72]:
mask_vacios = df_membresia.isna().all(axis=1)

if mask_vacios.any():
    indice_corte = mask_vacios.idxmax()
    df_membresia = df_membresia.iloc[:indice_corte]
else:
    print("No hay filas con todos los valores nulos.")

df_membresia.tail()

,Movil,Cuota,Tipo
18,3685.0,100.0,NaN
19,3767.0,100.0,NaN
20,2736.0,100.0,NaN
21,2849.0,100.0,NaN
22,3198.0,100.0,NaN


In [73]:
df_membresia['Tipo'].fillna('No', inplace=True)
df_membresia['Tipo'][df_membresia['Tipo'] == 'PREMIUN'] = 'Si'
df_membresia.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_9604\1827376845.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_membresia['Tipo'].fillna('No', inplace=True)


,Movil,Cuota,Tipo
0,3504.0,350.0,Si
1,7474.0,350.0,Si
2,3788.0,350.0,Si
3,4027.0,350.0,Si
4,9172.0,350.0,Si


In [74]:
df_membresia.head()

,Movil,Cuota,Tipo
0,3504.0,350.0,Si
1,7474.0,350.0,Si
2,3788.0,350.0,Si
3,4027.0,350.0,Si
4,9172.0,350.0,Si


In [75]:
df_membresia.to_csv("Output/df_membresia.csv", mode = 'a', header = False, index=False, encoding='utf-8-sig', sep=',')